In [21]:
import boto3
import pandas as pd
import json
import time
from botocore.exceptions import ClientError

# 1. Load your transcripts
df = pd.read_csv("call_transcripts.csv")
print(f"Loaded {len(df)} transcripts")

# 2. Connect to AWS Bedrock
bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# 3. Define a function to summarize
def summarize_transcript(transcript):
    prompt = f"""You are a 911 emergency response assistant analyzing call transcripts.
Summarize the key emergency details from this 911 call transcript.

Extract and format as follows:
- Emergency Type: [specific type of emergency]
- Location: [exact address if provided]
- People Involved: [caller name and any mentioned individuals] 
- Urgency Level: [High/Medium/Low based on severity]
- Key Details: [critical information for responders]

911 Call Transcript: {transcript}

Provide a concise, structured summary for emergency responders."""
    
    try:
        response = bedrock.invoke_model(
            modelId="anthropic.claude-3-haiku-20240307-v1:0",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 300,
                "temperature": 0.3,
                "messages": [
                    {
                        "role": "user", 
                        "content": prompt
                    }
                ]
            })
        )
        
        result = json.loads(response['body'].read())
        return result["content"][0]["text"]
        
    except Exception as e:
        print(f"Error processing transcript: {e}")
        return f"Error: {e}"

# 4. Test with first actual transcript (skip header)
# print("Processing first actual transcript as test...")
# first_transcript = df["Transcription"].iloc[0]  # This is now the first data row
# print(f"Processing: {first_transcript[:100]}...")

# summary = summarize_transcript(first_transcript)
# print("\nSUMMARY:")
# print("-" * 50)
# print(summary)
# print("-" * 50)
# print("Test complete!")

# To process all transcripts, uncomment the lines below:
print(f"\nProcessing all {len(df)} transcripts...")
df["Summary"] = df["Transcription"].apply(summarize_transcript)
df.to_csv("call_summaries.csv", index=False)
print("All summaries saved to call_summaries.csv")

Loaded 13 transcripts

Processing all 13 transcripts...
All summaries saved to call_summaries.csv


In [18]:

df.head()

,Transcription,Start Time,End Time,Duration (seconds),Location,Caller ID,Call Type
0,"911, what's your emergency? Caller: My husband...",2024-03-15 14:23:17,2024-03-15 14:28:42,325,"47284 Jennifer Islands, Port Kellychester, ND ...",Sarah Martinez,Chest pain
1,"Emergency services, what's happening? Caller: ...",2024-05-22 11:47:33,2024-05-22 11:52:18,285,"789 David Locks, Anthonyshire, CA 92103",Anonymous,Active shooter
2,"911 dispatch, what's your emergency? Caller: M...",2024-07-08 19:34:26,2024-07-08 19:41:15,409,"456 Maria Valley, Lake Christopherstad, TX 75201",Michael Chen,House fire
3,What's your emergency? Caller: I think my wife...,2024-01-19 09:15:44,2024-01-19 09:20:29,285,"123 Thompson Street, New Patricia, FL 33125",Robert Johnson,Stroke symptoms
4,"911, how can I help you? Caller: There's been ...",2024-09-03 16:42:11,2024-09-03 16:45:33,202,"890 Garcia Ranch, Williamsbury, IL 60614",Jennifer Lopez,Car accident (with injury)
